In [1]:
print("Ok")

Ok


In [2]:
%pwd

'd:\\End-to-end-Medical-Chatbot-Generative-AI-main\\End-to-end-Medical-Chatbot-Generative-AI-main\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'd:\\End-to-end-Medical-Chatbot-Generative-AI-main\\End-to-end-Medical-Chatbot-Generative-AI-main'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents


In [7]:
extracted_data=load_pdf_file(data='Data/')

In [8]:
len(extracted_data)

637

In [9]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [10]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [53]:
# text_chunks

In [11]:
from langchain.embeddings import HuggingFaceEmbeddings
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

embeddings = download_hugging_face_embeddings()
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))


C:\Users\lenovo\AppData\Local\Temp\ipykernel_8172\427521624.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
d:\End-to-end-Medical-Chatbot-Generative-AI-main\End-to-end-Medical-Chatbot-Generative-AI-main\medical-env311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Length 384


In [12]:
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [14]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot-1"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [15]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [16]:
from langchain.vectorstores import Pinecone

docsearch = Pinecone.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)


In [17]:
# Load Existing index 

from langchain.vectorstores import Pinecone
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = Pinecone.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)



In [18]:
docsearch

In [19]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [ ]:
retrieved_docs = retriever.invoke("What is Acne?")

In [28]:
retrieved_docs

[Document(metadata={'page': 441.0, 'source': 'Data\\Medical_book.pdf'}, page_content='in modern India and has been steadily gaining follow-\ners in the West.\nGALE ENCYCLOPEDIA OF MEDICINE 2428\nAviation medicine\nGEM - 0001 to 0432 - A  10/22/03 1:43 PM  Page 428'),
 Document(metadata={'page': 445.0, 'source': 'Data\\Medical_book.pdf'}, page_content='Care.” P.O. Box 3021, Quincy, CA 95971. <http://www.\ninsideayurveda.com>.\nDouglas Dupler\nAzithromycin see Erythromycins\nAZT see Antiretroviral drugs\nGALE ENCYCLOPEDIA OF MEDICINE 2432\nAyurvedic medicine\nGEM - 0001 to 0432 - A  10/22/03 1:43 PM  Page 432'),
 Document(metadata={'page': 608.0, 'source': 'Data\\Medical_book.pdf'}, page_content='feel the fetus kicking. Most women cannot tell what\nposition the fetus is in at any given moment.\nDiagnosis\nA health care provider can often tell the position of\nthe fetus by feeling it through the wall of the mother’s\nabdomen. Another clue to the position is the location\nwhere the heartbe

In [29]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

from langchain_openai import OpenAI
llm = OpenAI(temperature=0.2, max_tokens=500)

system_prompt = (
    """You are an assistant for question-answering tasks.
    i want you gives answers from these 
    """
    
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


response = rag_chain.invoke({"input": "where is pakistan"})
print(response["answer"])



System: Pakistan is located in South Asia, bordering India to the east, Afghanistan and Iran to the west, and China to the north. It also has a coastline along the Arabian Sea to the south.


In [24]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [73]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])



Acromegaly and gigantism are disorders caused by an abnormal release of a chemical from the pituitary gland in the brain, leading to increased growth in bone and soft tissue. This can result in a variety of disturbances throughout the body, including unusual height. Acromegaly is a relatively rare disorder that is often not diagnosed until middle age due to the gradual onset of symptoms.


In [26]:
response = rag_chain.invoke({"input": "Where is pakistan located?"})
print(response["answer"])



Pakistan is located in South Asia, bordered by India, Afghanistan, Iran, and China. It is a predominantly Muslim country and gained independence from British rule in 1947. Its capital is Islamabad and its largest city is Karachi.
